In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import ipywidgets as widgets

# Read the CSV file
df = pd.read_csv('../ring_simulation_results.csv')

df = df[df['N'] == 128]

def update_plot(amplitude_threshold):
    # Group by A and relative_w1, then count bumps
    bump_counts = df.groupby(['A', 'relative_w1']).agg({
        'amplitude': lambda x: (x > amplitude_threshold).sum(),
        'critical_w1': 'first'  
    }).reset_index()
    bump_counts.columns = ['A', 'relative_w1', 'bump_count', 'critical_w1']

    # Calculate actual w1 values
    bump_counts['w1'] = bump_counts['critical_w1'] + bump_counts['relative_w1']

    # Create the scatter plot
    plt.figure(figsize=(12, 8))

    scatter = plt.scatter(bump_counts['A'], bump_counts['w1'], 
                          c=bump_counts['bump_count'], s=bump_counts['bump_count']*5,
                          cmap='viridis', alpha=0.7)

    plt.colorbar(scatter, label='Number of Bumps Formed')

    # Plot the critical w1 line using values from the dataset
    A_values = bump_counts['A'].unique()
    critical_w1_values = bump_counts.groupby('A')['critical_w1'].first()
    plt.plot(A_values, critical_w1_values, label='Critical $w_1$', color='red', linewidth=2)

    plt.title(f'Number of Bumps Formed for Different A-w1 Pairs\nAmplitude Threshold: {amplitude_threshold:.2f}')
    plt.xlabel('A (Noise Amplitude)')
    plt.ylabel('$w_1$')
    plt.legend(loc='upper right')

    plt.ylim(bump_counts['w1'].min() - 0.1, bump_counts['w1'].max() + 0.1)
    plt.xlim(A_values.min() - 0.5, A_values.max() + 0.5)

    plt.tight_layout()
    plt.show()

    # Print summary statistics
    print(bump_counts.groupby('A')[['bump_count']].describe())

    # Optional: Print a sample of the data
    print("\nSample of bump count data:")
    print(bump_counts.head(10))

# Create an interactive widget
interact(update_plot, 
         amplitude_threshold=FloatSlider(min=0, max=2, step=0.1, value=0.5, 
                                         description='Amplitude Threshold:',
                                         style={'description_width': 'initial'},
                                         layout=widgets.Layout(width='50%')))

# Print r_0 value
print(f"r_0: {df['critical_w1'].iloc[0] + np.sqrt((2*np.pi/256)*(df['A'].iloc[0]))}")


interactive(children=(FloatSlider(value=0.5, description='Amplitude Threshold:', layout=Layout(width='50%'), m…

r_0: 3.9348680723879


In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from scipy import stats

# Load the data
df = pd.read_csv('../ring_simulation_results.csv')

# Filter for N=128
df = df[df['N'] == 128]

# Create unique sorted lists of relative_w1 and A values
relative_w1_values = sorted(df['relative_w1'].unique())
A_values = sorted(df['A'].unique())

def plot_distribution(A_index, relative_w1_index):
    A = A_values[A_index]
    relative_w1 = relative_w1_values[relative_w1_index]
    
    # Filter data for the selected A and relative_w1
    filtered_df = df[(df['A'] == A) & (df['relative_w1'] == relative_w1)]
    
    if filtered_df.empty:
        print(f"No data available for A={A} and relative_w1={relative_w1}")
        return
    
    # Plot histogram and KDE
    fig, ax = plt.subplots(figsize=(12, 6))
    
    # Histogram
    ax.hist(filtered_df['amplitude'], bins=20, density=True, alpha=0.7, color='skyblue', edgecolor='black')
    
    # KDE
    kde = stats.gaussian_kde(filtered_df['amplitude'])
    x_range = np.linspace(filtered_df['amplitude'].min(), filtered_df['amplitude'].max(), 100)
    ax.plot(x_range, kde(x_range), 'r-', linewidth=2, label='KDE')
    
    ax.set_title(f'Amplitude Distribution for A={A:.2f}, relative_w1={relative_w1:.2f}')
    ax.set_xlabel('Amplitude')
    ax.set_ylabel('Density')
    
    # Add vertical line for mean
    mean_amplitude = filtered_df['amplitude'].mean()
    ax.axvline(mean_amplitude, color='g', linestyle='dashed', linewidth=2, label=f'Mean: {mean_amplitude:.4f}')
    
    ax.legend()
    plt.tight_layout()
    plt.show()
    
    # Print some statistics
    print(f"Number of data points: {len(filtered_df)}")
    print(f"Mean amplitude: {mean_amplitude:.6f}")
    print(f"Standard deviation: {filtered_df['amplitude'].std():.6f}")
    print(f"Critical w1: {filtered_df['critical_w1'].iloc[0]:.6f}")
    print(f"Actual w1: {filtered_df['w1'].iloc[0]:.6f}")

# Create interactive widgets
A_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(A_values) - 1,
    step=1,
    description='A index:',
    continuous_update=False,
    orientation='horizontal'
)

relative_w1_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(relative_w1_values) - 1,
    step=1,
    description='relative_w1 index:',
    continuous_update=False,
    orientation='horizontal'
)

# Create labels to show actual values
A_label = widgets.Label()
relative_w1_label = widgets.Label()

def update_labels(A_index, relative_w1_index):
    A_label.value = f"A: {A_values[A_index]:.2f}"
    relative_w1_label.value = f"relative_w1: {relative_w1_values[relative_w1_index]:.2f}"

# Use interactive to update labels when sliders change
interactive(update_labels, A_index=A_slider, relative_w1_index=relative_w1_slider)

# Display widgets
display(A_label, relative_w1_label)

# Create the main interactive plot
interact(plot_distribution, A_index=A_slider, relative_w1_index=relative_w1_slider);

Label(value='A: 0.00')

Label(value='relative_w1: -0.20')

interactive(children=(IntSlider(value=0, continuous_update=False, description='A index:', max=9), IntSlider(va…